In [353]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 
from typing import Any, Optional, List

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger
from qubx.data.readers import CsvStorageDataReader, MultiQdbConnector, QuestDBConnector, DataReader, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC, AsPandasFrame, RestoreTicksFromOHLC, AsOhlcvSeries
from qubx.pandaz.utils import *
from qubx.data.helpers import  InMemoryCachedReader

 >  [dev] installing cython rebuilding hook
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Aux Data interface

In [20]:
r_csv = CsvStorageDataReader("../tests/data/csv")
r_mqc = MultiQdbConnector("xlydian-data")
r_qdb = QuestDBConnector(host="xlydian-data")

In [10]:
r_csv.get_aux_data_ids(), r_qdb.get_aux_data_ids(), r_mqc.get_aux_data_ids()

(set(),
 {'average_quote_volume', 'candles', 'fundamental_data', 'symbols'},
 {'average_quote_volume', 'candles', 'fundamental_data', 'symbols'})

In [ ]:
r_mqc.get_aux_data('candles', exchange='BINANCE.UM', 
                   symbols=['BTCUSDT', 'ETHUSDT'], 
                   start='2024-01-01', stop='2024-01-31')

In [26]:
r_qdb.get_aux_data('fundamental_data', exchange='BINANCE.UM', 
                #    symbols=['BTCUSDT', 'ETHUSDT'], 
                   start='2024-01-01', stop='2024-02-01')

metric                  market_cap       price        volume    volume_usd
timestamp  symbol                                                         
2024-01-01 1INCHUSDT  4.810212e+08     0.42810  2.528007e+07  1.082240e+07
           AAVEUSDT   1.604350e+09   108.78000  1.155772e+06  1.257249e+08
           ACHUSDT    1.051147e+08     0.02136  4.327358e+08  9.243236e+06
           ADAUSDT    2.076801e+10     0.59410  3.323489e+08  1.974485e+08
           AGIXUSDT   3.989788e+08     0.31970  4.189333e+07  1.339330e+07
...                            ...         ...           ...           ...
2024-01-31 YFIUSDT    2.435439e+08  7309.00000  1.259769e+03  9.207652e+06
           ZECUSDT    3.307871e+08    22.59000  5.673166e+05  1.281568e+07
           ZENUSDT    1.160340e+08     7.94800  5.515012e+05  4.383332e+06
           ZILUSDT    3.599965e+08     0.02046  3.587373e+08  7.339764e+06
           ZRXUSDT    2.798377e+08     0.33150  1.322362e+08  4.383631e+07

[5456 rows x 4 columns]

# Handy reader

In [354]:
F = '4h'
_P = lambda ds: [print(f"{s} : {len(v)} | {v.index[0]} : {v.index[-1]}") for s, v in ds.items()]
mcr = InMemoryCachedReader('BINANCE.UM', r_mqc, F)

In [356]:
_P(mcr._get_smbs_at(['LTCUSDT', 'BTCUSDT'], "2023-01-01", "2024-01-01" ));

LTCUSDT : 2190 | 2023-01-01 00:00:00 : 2023-12-31 20:00:00
BTCUSDT : 2190 | 2023-01-01 00:00:00 : 2023-12-31 20:00:00


In [357]:
_P(mcr._get_smbs_at(['LTCUSDT', 'BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'XRPUSDT'], "2021-01-01", "2024-06-01" ));

LTCUSDT : 7452 | 2021-01-01 00:00:00 : 2024-05-31 20:00:00
BTCUSDT : 7458 | 2021-01-01 00:00:00 : 2024-05-31 20:00:00
ETHUSDT : 7482 | 2021-01-01 00:00:00 : 2024-05-31 20:00:00
SOLUSDT : 7452 | 2021-01-01 00:00:00 : 2024-05-31 20:00:00
XRPUSDT : 7452 | 2021-01-01 00:00:00 : 2024-05-31 20:00:00


In [358]:
mcr1 = InMemoryCachedReader('BINANCE.UM', r_mqc, F)
_P(x1:=mcr1._get_smbs_at(['LTCUSDT'], "2021-01-01", "2024-06-01"))
_P(x2:=mcr._get_smbs_at(['LTCUSDT'],  "2021-01-01", "2024-06-01"))
all(x1['LTCUSDT']==x2['LTCUSDT'])

LTCUSDT : 7452 | 2021-01-01 00:00:00 : 2024-05-31 20:00:00
LTCUSDT : 7452 | 2021-01-01 00:00:00 : 2024-05-31 20:00:00


True

In [409]:
class X:
    def __getitem__(self, keys):
        _start, _stop = None, None
        _instruments = []
        _as_dict = False
        if isinstance(keys, (tuple)):
            for k in keys:
                if isinstance(k, slice):
                    _start, _stop = k.start, k.stop
                if isinstance(k, (list, tuple)):
                    _instruments = k
                    _as_dict = True
                if isinstance(k, str):
                    _instruments.append(k)
        else:
            if isinstance(keys, (list, tuple)):
                _instruments.extend(keys)
                _as_dict = True
            else:
                _instruments.append(keys)
        # _as_dict 
        print(f" >>> Get data for {_instruments} from {_start} to {_stop} | {_as_dict}")

s = X()
s[["BTCUSDT"], '2023-01-01':'2024-01-01' ]
s["BTCUSDT", '2023-01-01':'2024-01-01' ]
s[['BTC']]
s['BTC']
# s[{'b': 1}]

 >>> Get data for ['BTCUSDT'] from 2023-01-01 to 2024-01-01 | True
 >>> Get data for ['BTCUSDT'] from 2023-01-01 to 2024-01-01 | False
 >>> Get data for ['BTC'] from None to None | True
 >>> Get data for ['BTC'] from None to None | False
